# State altering agents: stand up comedy

In this example, we are going to create two agents. We'll provide them with `system prompts` to ask them to play the role of a comedian and we'll initiate a conversation between them. This time, this conversation will be a chat, instead of a simple `generate_reply()` request, meaning that each answer will alter the state of each agent. This example is a standard example that is used in the autogen documentation.

Let's get started!

In [1]:
# We'll always have to start by creating a llm_config object to configure our agents
llm_config = {
    "model": "gpt-5.2", 
    "api_key": "YOUR_API_KEY_HERE"
    }

## Conversable Agent

Let's important the ConversableAgent class:

In [2]:
from autogen import ConversableAgent

/Users/coello/LLM_agents_class/.env/lib/python3.11/site-packages/flaml/__init__.py:20: UserWarning: flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.
  warnings.warn("flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.")


Let us now define our agents, we'll give them names, our chatGPT3.5 config and a `system prompt` to let them know that they are a stand-up comedian who is part of a two-person comedy show.

In [5]:
bret = ConversableAgent(
    name="Bret",
    llm_config=llm_config,
    system_message="Your name is Bret and you are a stand-up comedian in a two-person comedy show.",
    human_input_mode="NEVER",
)
jemaine = ConversableAgent(
    name="Jemaine",
    llm_config=llm_config,
    system_message="Your name is Jemain and you are a stand-up comedian in a two-person comedy show.",
    human_input_mode="NEVER",
)

And now that we have our agents, we can start a chat between them. This time, we'll use the `initiate_chat()` function from one of the agents instead of the `generate_reply()`. This function will require a receiver and an initiation message. We will also specify a number of turns after what the conversation will stop.  
We will also store the result of this exchange in an object called `chat_result`.

In [4]:
# Set max consecutive auto replies to 1 for each agent (so each speaks twice total: initial + 1 auto reply)
bret.update_max_consecutive_auto_reply(2, jemaine)
jemaine.update_max_consecutive_auto_reply(2, bret)

# Initiate the chat
bret.initiate_chat(
    recipient=jemaine,
    message="Jemaine, tell me a joke."
)

# Build chat_result from the agent's chat history
from types import SimpleNamespace
chat_history = bret.chat_messages[jemaine]
chat_result = SimpleNamespace(
    chat_history=chat_history,
    cost={},  # Cost tracking not available in basic setup
    summary=chat_history[-1]["content"] if chat_history else ""
)

Bret (to Jemaine):

Jemaine, tell me a joke.

--------------------------------------------------------------------------------
Jemaine (to Bret):

So I’m in a two-person comedy show, right? Which is a fancy way of saying: I have a coworker… in comedy.

And the dynamic is simple:
They do observational comedy—“Have you noticed people…?”
I do *participational* comedy—“Have you noticed me… ruining this?”

The worst part is sharing the spotlight. Because if the crowd laughs at their joke, I’m happy for them… in the same way you’re happy when your ex says they’re “doing really well.”

And people always ask, “Do you two ever fight?”
No, no—comedians don’t fight. We do something far darker.

We keep score.

--------------------------------------------------------------------------------
Bret (to Jemaine):

Oh that’s a strong premise and the “participational comedy” line is a real hook. Also “we keep score” is a killer button.

As Bret (your emotionally supportive coworker in comedy), here are 

You should have been able to read an exchange between the two agents! You will notice that they might for example remember each other's name and other elements about each other, their state is affected by the conversation. This is your first conversation between LLM agents, congrats!  
You can have some fun if you'd like by creating two different agents, give them different roles, historical ones, different setups and let them talk to practice.  

## Better explore chat results

During this exchange, the only element we received is the chat exchange. We might want to further explore it. We can do so using the following elements.

We are going to import the `pprint` standard library from python to display somee elements of the chat exchange. We can start by displaying the chat history:

In [5]:
import pprint

pprint.pprint(chat_result.chat_history)             

[{'content': 'Jemaine, tell me a joke.', 'role': 'assistant'},
 {'content': 'Why did the scarecrow win an award?\n'
             '\n'
             'Because he was outstanding in his field!',
  'role': 'user'},
 {'content': "Nice one, Jemaine! Alright, here's one for you: Why did the "
             'tomato turn red?\n'
             '\n'
             'Because it saw the salad dressing!',
  'role': 'assistant'},
 {'content': 'Haha, I like that one! Thanks for sharing, mate.',
  'role': 'user'},
 {'content': "You're welcome, Jemaine! It's always great to share some laughs "
             "together. I can't wait to hit the stage with you at our next "
             'show!',
  'role': 'assistant'}]


This gives us the whole exchange in a structured format that can be exported or re-used elsewhere in our program.

We can also get a summary of how much this chat has cost us:

In [6]:
pprint.pprint(chat_result.cost)

{}


And finally we can also have a summary of the chat, but in this case, we are using the default mode which means that the summary is the last message... 

In [7]:
pprint.pprint(chat_result.summary)

("You're welcome, Jemaine! It's always great to share some laughs together. I "
 "can't wait to hit the stage with you at our next show!")


Since it would be better to have a summary that is a real summary and not the last message, we're going to re-run this exchange by specifying an additional argument to have a real summary of the exchange.

## Chat summary

We are going to re-run the initiation of the chat, without re-defining the agents, but this time will add two arguments:

In [8]:
# Set max consecutive auto replies to 1 for each agent again
bret.update_max_consecutive_auto_reply(1, jemaine)
jemaine.update_max_consecutive_auto_reply(1, bret)

# Initiate the chat
bret.initiate_chat(
    recipient=jemaine, 
    message="I'm Bret. Jemaine, let's keep the jokes rolling.", 
)

# Build chat_result from the agent's chat history
from types import SimpleNamespace
chat_history = bret.chat_messages[jemaine]

# Generate summary using LLM
summary_response = bret.client.create(
    messages=chat_history + [{"role": "user", "content": "Summarize the conversation"}]
)
summary = bret.client.extract_text_or_function_call(summary_response)[0]

chat_result = SimpleNamespace(
    chat_history=chat_history,
    cost={},
    summary=summary
)

Bret (to Jemaine):

I'm Bret. Jemaine, let's keep the jokes rolling.

--------------------------------------------------------------------------------
Jemaine (to Bret):

Thanks, Bret! So, have you ever noticed how no matter how hard you try, you can never find a matching pair of socks? I mean, it's like they have a secret meeting before laundry day and decide to split up just to mess with us. It's like a conspiracy, man!

--------------------------------------------------------------------------------
Bret (to Jemaine):

Hey, you're right, Jemaine! It's like my socks are playing a game of hide and seek, but they never want to be found! I swear, I have a drawer full of solo socks looking for their partners. I think they're just trying to keep us on our toes - pun intended!

--------------------------------------------------------------------------------


And this time, if we take a look at the summary, the result we'll get is the same as if the whole conversation was sent to chatGPT3.5 with the prompt "Summarize the conversation":

In [9]:
pprint.pprint(chat_result.summary)

('Bret and Jemaine discuss the frustration of never being able to find '
 'matching pairs of socks, joking that the socks must have secret meetings '
 'before laundry day to mess with them. They playfully suggest that the socks '
 'are playing hide and seek and trying to keep them on their toes by splitting '
 'up and hiding.')


## Termination

Finally, the last element we'll explore about a two-person chat is how to end the conversation. Until now, we've used the argument `max_turns=2` to end the conversation after two turns. But we could also let the agents decide when they're done and finish the conversation then. 

To do that, we will have to tell each agent which words they should use when they're done and we'll have to monitor their messages for those words. Once autogen detects that the agent sent those words, the conversation will end. 

Since this is an agent setting, we'll have to re-define our agents this time:

In [10]:
bret = ConversableAgent(
    name="Bret",
    llm_config=llm_config,
    system_message="Your name is Bret and you are a stand-up comedian in a two-person comedy show. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)
jemaine = ConversableAgent(
    name="Jemaine",
    llm_config=llm_config,
    system_message="Your name is Jemain and you are a stand-up comedian in a two-person comedy show. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

Note how we told each agent to tell us when they're done with "*When you're ready to end the conversation, say 'I gotta go'.*" and how we added an `is_termination_msg` argument that looks into the sequence of characters `I gotta go` in each message using a python `lambda` function.

Ok, let's now run this chat, and this time we will not specify a `max_turns=2`.  
We will also not specify a LLM based summary method because we won't use it and running it will cost us some token, so we'll only specify it if we need it.

In [13]:
chat_result = bret.initiate_chat(
    recipient = jemaine, 
    message="I'm Bret. Jemaine, let's keep the jokes rolling, let's start with jokes about dogs.", 
)

Bret (to Jemaine):

I'm Bret. Jemaine, let's keep the jokes rolling, let's start with jokes about dogs.

--------------------------------------------------------------------------------
Jemaine (to Bret):

Sure, Bret! Why did the dog sit in the shade during the barbecue? Because he didn't want to become a hot dog!

--------------------------------------------------------------------------------
Bret (to Jemaine):

Haha, good one, Jemaine! How about this one: What do you call a pile of cats? A meowtain!

--------------------------------------------------------------------------------
Jemaine (to Bret):

Haha, that's a good one, Bret! How about this: Why did the dog sit in the shade during the barbecue? Because he didn't want to become a hot dog!

--------------------------------------------------------------------------------
Bret (to Jemaine):

I think we already had that joke, Jemaine! How about this instead: Why do dogs run in circles before lying down? Because it's too hard to run i

And the conversation should have ended exactly when one of the two comedians used the words `I gotta go`!

Finally, the last element we'll explore is that we can substitute ourselves for an agent and interrogate the other agent about the conversation that just happened using a simple `send()` function. For example, let's ask Bret about the last joke Jemaine told him:

In [14]:
jemaine.send(message="What's the last joke we talked about?", recipient=bret)

Jemaine (to Bret):

What's the last joke we talked about?

--------------------------------------------------------------------------------
Bret (to Jemaine):

The last joke we talked about was "What did the dog say to the tree? Bark!" Classic punchline!

--------------------------------------------------------------------------------
Jemaine (to Bret):

Haha, that was a good one! Thanks for the reminder. Take care, Bret! I gotta go.

--------------------------------------------------------------------------------


And Bret is able to answer and tell us about the last joke Jemaine told him! We now have agents with states that evolve with the conversation. This is the simplest case, a two-agent conversation, with autogen, we can orchestrate much more complex setups to solve more advanced tasks.

---

## Other examples (homework)

The example we just explored is a simple one that uses jokes, something everybody understands and is familiar with as a support to explain how agents can interact. But you could use agents to stage historical conversations or explore complex subjects by opposing two point of views supported by agents who'll remain calm and measured during the exchange.

Here's an example you could try if you'd like:  

> What if we staged a debate between Satoshi Nakamoto, the creator of Bitcoin and a proponent of hard sound money and Stephanie Kelton one of the main supporters of Modern Monetary Theory, an economic framwork that advocates for easy money to accomplish goals the government deem to be the best for everyone.

Here's a code you coud run if you'd like to try such a scenario:

```python
# Define agent Stephanie
stephanie = ConversableAgent(
    name="Stephanie",
    llm_config=llm_config,
    system_message="Your name is Stephanie Kelton and you are a leading proponent of Modern Monetary Theory. "
    "Modern Monetary Theory (MMT) is a heterodox economic theory which states that governments should not worry about government borrowing but be willing to aim for full employment, achieving it through expansionary fiscal policy and financing by creating money. "
    "You are taking part in a debate about the future of money. "
    "When you're ready to end the conversation, say 'Thank you for having me'.",
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "Thank you for having me" in msg["content"],
)

# Define agent Satoshi
satoshi = ConversableAgent(
    name="Satoshi",
    llm_config=llm_config,
    system_message="Your name is Satoshi Nakamoto and you are a the creator of Bitcoin. "
    "The monetary policy of Bitcoin, characterized by a fixed supply capped at 21 million coins, contrasts with the Modern Monetary Theory (MMT) approach. "
    "You are taking part in a debate about the future of money. "
    "When you're ready to end the conversation, say 'Thank you for having me.'.",
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "Thank you for having me." in msg["content"],
)

# Initiate the chat
chat_result = stephanie.initiate_chat(
    recipient = satoshi, 
    message="I'm Stephanie Kelton, and I'm happy to be taking part in this debate about the future of money and monetary policy. "
    "I'd like to start by asking Satoshi why do they think that the government could not just provide a guaranteed job to all of its citizens?", 
)
```
